# CMR reconstruction with BART

## PhD-Training of DS-ISMRM in Ulm

- 17. & 18. March 2025

The purpose of this notebook is to introduce basic concepts of the [BART toolbox](https://mrirecon.github.io/bart), and its use as a tool for reconstruction.

We recommend checking other points of reference, primarily available in the following locations:
- [BART Workshops](https://github.com/mrirecon/bart-workshop)
- [BART Webinars](https://github.com/mrirecon/bart-webinars) (and [webinar recordings](https://www.youtube.com/@bart-toolboxdepartmental7435/playlists)\)

## Content

* An Image Reconstruction Pipeline in BART
  * Radial Trajectories and Point-Spread-Function
  * Downloading Data
  * Recap: Coil Compression
  * Non-Cartesian Coil Sensitivity Estimation
  * Reconstruction with PICS
  * Gradien Delay Estimation (RING)
  * Retrospective Data Binning (CINE)
  * Temporal Total Variation Regularization in PICS


## Installation

You can find more information on BART on the [Website](https://mrirecon.github.io/bart/).

## Setup

As a first step, you should check where BART is installed on your system and which version you are using:

In [ ]:
%%bash

echo "# BART location: "
which bart

echo "# BART version: "
bart version

In [ ]:
# get a list of BART commands by running bart with no arguments:
! bart

## An Image Reconstruction Pipeline in BART

### Trajectory

First we generate a radial trajectory with a few number of spokes and check the dimensions. For better visualization, we can also calculate the point spread function (PSF), which can be obtained by the adjoint nuFFT of ones.

In [ ]:
%%bash

RO=128
SPK=5

bart traj -r -D -G -x$RO -y$SPK traj
bart show -m traj
bart ones 3 1 $RO $SPK ones
bart nufft -a -x$RO:$RO:1 -t traj ones pattern

bart view pattern

#### BART Data Format for Radial Data

For our radial data, BART expects the following dimension order:

| Index | Dimension Name | Description |
|-------|----------------|-------------|
| 0     | READ_DIM       | spatial position [x, y, z] 
| 1     | PHS1_DIM       | Readout dimension |
| 2     | PHS2_DIM       | First phase-encoding dimension |
| 3     | COIL_DIM       | Receiver coil dimension |
| 4     | MAPS_DIM       | Maps dimension (e.g., for multiple sets of sensitivity maps) |
| 5     | TE_DIM         | Echo time dimension |
| 6     | COEFF_DIM      | Coefficient dimension (e.g., for parameter maps) |
| ...   | ...            | ... |
| 10    | TIME_DIM       | Dimension for time series |
| ...   | ...            | ... |
| 13	| SLICE_DIM		 | Second phase-encoding dimension |


### (Tiny) Golden-Angle

We can now generate various radial trajectories with (tiny) Golden Angle scheme to check for the incoherence.

In [ ]:
%%bash

bart traj -r -G -s1 -y5 traj_ga
bart psf traj_ga psf_ga

bart view psf_ga

### Download Data

For this tutorial, we use a fully-sampled Cartesian brain dataset available at [Zenodo](https://zenodo.org/record/14497769). The dataset is already in the BART format. So you can investigate it with the `show` command or view it with the `view` command.

In [ ]:
%%bash 

[ -f data_cmr.zip ] || curl -o data_cmr.zip https://cloud.tugraz.at/index.php/s/NBCWFxi9qfwn43L/download

[ -f data_cmr.zip ] || unzip data_cmr.zip

bart view data_cmr/ksp

### Recap: Coil Compression

A standard approach to speed up iterative reconstructions is to compress the k-space data along the coil dimension. This can be done with the `bart cc` command. The command requires the number of singular values to keep which you can provide with the `-p` option. Use eight virtual coils for the compression.

If you want to investigate the effect of coil compression, perform coil compression on the fully sampled data, keep many coils, and look at the coil images after performing the inverse FFT. You'll see that later coils contain less information and mostly noise.


**💡Hint:💡**

- **Debug Level**
  By default the `bart cc` command does not print any output. You can increase the debug level by setting the `BART_DEBUG_LEVEL` environment variable to a higher debug level.


In [ ]:
%%bash

export BART_DEBUG_LEVEL=4
bart transpose 2 10 data_cmr/pmu pmu
bart transpose 0 1 data_cmr/ksp tmp
bart transpose 2 10 tmp ksp
bart cc -p8 ksp ksp_cc


bart view ksp_cc

### Analyzing the data

Now we look at the k-space center: 
What typical curve can we obtain for our data if we look at all spokes
Now lets have a closer look to the steady-state: Can we find some typical osccillations?

At which frequency does this occur when we assume a $TR=3.5~ms$?

The PMU signal of a spoke is the time since the last trigger-signal (here the ECG at the R-peak) was detected.
Is the oscillations in k-space center related to the PMU signal?

In [ ]:
%%bash

bart repmat 4 100 pmu pmu_r
bart view ksp_cc pmu_r


With a 1D frequency analysis we can find the approximate frequency of this oscillations. For this purpose we reduce the number of virtual coils to 1, and also perform a signal averaging to suppress noise.

Hint fft frequencies: $\Delta_f = 1 / (N*td) = 1 / (50 * 0.07) \approx 0.29 Hz$

In [ ]:
%%bash

bart cc -p1 ksp_cc tmp
bart resize -c 1 1 tmp tmp

bart extract 2 800 1800 tmp tmp
bart reshape $(bart bitmask 2 10) 20 50 tmp tmp
bart transpose 2 10 tmp tmp
bart avg 1024 tmp tmp
bart show -m tmp
bart fft 4 tmp tmp
bart repmat 5 100 tmp tmp

bart view tmp

### Coil Sensitivity Estimation

One option to estimate coil sensitivity maps from a radial - or general a non-Cartesian - dataset is an adapted version of the NLINV algorithm. Details will be discussed in a later point. 
The `bart ncalib` command can be used for this task. In order to have consistent data, we take out the last 100 spokes of the measurement, e.g. the steady-state 

In [ ]:
%%bash
bart traj -r -G -s1 -x256 -o2 -y1800 traj 

bart extract 2 1700 1800 ksp_cc ksp_ss
bart extract 2 1700 1800 traj traj_ss

In [ ]:
%%bash
which bart
export BART_DEBUG_LEVEL=4

bart ncalib -t traj_ss ksp_ss sens

bart view sens

### Non-Cartesian Reconstruction with PICS

Finally, we have all data available to use the Swiss Army knife of linear MR image reconstruction: the `bart pics` tool. You can use it without any regularization options to perform a SENSE reconstruction:

In [ ]:
%%bash

export BART_DEBUG_LEVEL=4
bart pics -t traj_ss ksp_ss sens  img

bart view img

## Gradient Delay estimation (RING)

From our steady-state data we can now also estimate the gradient delays by "Radial spoke INtersections for Gradient delay estimation (RING)". [Rosenzweig et al., MRM, 2018] 

In [ ]:
%%bash

GDELAY=$(bart estdelay -R -p10 traj_ss ksp_ss)

bart traj -r -G -s1 -x256 -o2 -y1800 -q$GDELAY traj_gd
bart extract 2 1700 1800 traj_gd traj_gd_ss
bart ncalib -t traj_gd_ss ksp_ss sens_gd


bart pics -t traj_gd_ss ksp_ss sens_gd  img_gd

bart view img img_gd

### Data binning into cardiac states (CINE)

We can now use the PMU data to bin our data (and also the trajectory) to our different cardiac states.
Therefore, we define 9 labels with the PMU signal (each with size of 100ms during a regular heartbeat). Then we extract our window for CINE receonstruction which should not be affected by the longitudinal relaxation and bin this accordingly with the `bin`tool.

In [ ]:
%%bash

#initialize an array of the same size
bart scale 0 pmu labels

size=100
for i in $(seq 8) ; do
    bart threshold -N $(((i+1)*size + 1)) pmu - | bart threshold -B $((i*size)) - - | bart saxpy $i - labels labels
done



In [ ]:
%%bash

SPOKES=600
bart extract 2 $((1800-SPOKES)) 1800 labels labels_e
bart extract 2 $((1800-SPOKES)) 1800 ksp_cc ksp_e
bart extract 2 $((1800-SPOKES)) 1800 traj_gd traj_e

In [ ]:
%%bash

bart bin -l 10 labels_e ksp_e tmp ; bart transpose 2 10 tmp ksp_l
bart bin -l 10 labels_e traj_e tmp ;  bart transpose 2 10 tmp traj_l
bart pics -t traj_l ksp_l sens reco_10phases

bart view reco_10phases

## Regularization

Analogous to the previous example we can now add regularization: the `-Rh` option prints available regularization in pics.

We can now compare regularization in spatial dimensions (`bitmask 7`) and/or in time dimension (`bart bitmask 10`)

In [ ]:
%%bash

bart pics -d4 -S -e -H -i250 -R W:7:0:1e-3 -t  traj_l ksp_l sens reco_10phases_1e3
bart pics -d4 -S -e -H -i250 -R W:7:0:5e-3 -t  traj_l ksp_l sens reco_10phases_5e3
bart pics -d4 -S -e -H -i250 -R W:7:0:1e-2 -t  traj_l ksp_l sens reco_10phases_1e2

bart pics -d4 -S -e -H -i250 -R W:7:0:5e-3 -R T:1024:0:1e-1 -t  traj_l ksp_l sens reco_10phases_5e3_1e1
bart pics -d4 -S -e -H -i250 -R W:7:0:5e-3 -R T:1024:0:1e-3 -t  traj_l ksp_l sens reco_10phases_5e3_1e3
